# Tutorial

## Declaration problem and solving

### [(Non) Linear Problem](https://flopt.readthedocs.io/en/latest/tutorial/nlp.html#non-linear-problem)

The problem we solve is as follows.

```
minimize  2*(3*a+b)*c**2 + 3
s.t.      0 <= a <= 1, a is integer
          1 <= b <= 2, b is continuous
          1 <= c <= 3, c is continuous
```

In [ ]:
import flopt

create variables a, b and c

In [ ]:
a = flopt.Variable(name="a", lowBound=0, upBound=1, cat="Integer")
b = flopt.Variable(name="b", lowBound=1, upBound=2, cat="Continuous")
c = flopt.Variable(name="c", lowBound=1, upBound=3, cat="Continuous")

create problem and set objective function

In [ ]:
prob = flopt.Problem(name="Test")

# set objective function
prob += 2*(3*a+b)*c**2+3

run solver with standard output by solver

In [ ]:
status, logs = prob.solve(timelimit=1, msg=True)

status of solution

In [ ]:
print(status)

get best objective function value

In [ ]:
print("obj value = ", prob.getObjectiveValue())

get variable's value of that best solution

In [ ]:
print("a =", a.value())  # flopt.Value(a)
print("b =", b.value())
print("c =", c.value())

In [ ]:
# show the transition of objetive values of incumbent solutions
fig, ax = logs.plot(label="objective value of best solution", marker="o", linestyle="--")

When you select algorithm to solve problem, you create a Solver object and specify it as solver parameter in problem.solve().

In [ ]:
solver = flopt.Solver(algo="Random")  #  searching algorithm selected by flopt
prob.solve(solver=solver, timelimit=1, msg=1)

### [(Non) Linear Problem with Constraints](https://flopt.readthedocs.io/en/latest/tutorial/nlp_constraint.html#non-linear-problem-with-constraints)

```
minimize  2*(3*a+b**2) + 3
s.t.      a * b >= 2
          a + b >= 2
          0 <= a <= 1, a is integer
          1 <= b <= 2, b is continuous
```

We have constraints a * b >= 2 and a +b >= 2.

In [ ]:
import flopt

In [ ]:
# variables
a = flopt.Variable("a", 0, 1, cat="Continuous")
b = flopt.Variable("b", 1, 2, cat="Continuous")

# problem
prob = flopt.Problem(name="Test")
prob += 2*(3*a+b**2)+3  # set the objective function
prob += a*b >= 2      # add constraint
prob += a+b >= 2      # add constraint

# solver setting
solver = flopt.Solver(algo="Scipy")  # select the scipy function
solver.setParams(n_iter=10)  # select the scipy function

# run solver
prob.solve(solver, timelimit=10)

# get best solution
print("obj value　=", prob.getObjectiveValue())
print("a =", a.value())
print("b =", b.value())

show the details of problem

In [ ]:
prob.show()

get the solvers that can solve this problem

In [ ]:
flopt.allAvailableSolvers(prob)

#### AutoSearch (default solver)

select appropriate solver for the problem

In [ ]:
# variables
a = flopt.Variable("a", 0, 1, cat="Continuous")
b = flopt.Variable("b", 1, 2, cat="Continuous")

# problem
prob = flopt.Problem(name="Test")
prob += 2*a + 3*b # set the objective function

In [ ]:
# solver setting
solver = flopt.Solver(algo="auto")
solver.setParams(timelimit=1)
solver.select(prob).name

In [ ]:
flopt.allAvailableSolvers(prob)

### [Black Box Optimization Problem](file:///Users/tateiwa/Documents/flopt_dev/docs/_build/html/tutorial/black_box.html#black-box-optimization-problem)

```
minimize  simulator(a, b)
s.t.      0 <= a <= 1, a is integer
          1 <= b <= 2, b is continuous
```

In [ ]:
import flopt

In [ ]:
# variables
a = flopt.Variable(name="a", lowBound=1, upBound=3, cat="Integer")
b = flopt.Variable(name="b", lowBound=1, upBound=2, cat="Continuous")

# problem
prob = flopt.Problem(name="CustomExpression")

create CustomExpression to represetn blackbox function.<br>
In this tutorial, we use simple function as blackbox.

In [ ]:
def blackbox(a, b):
    import math
    return math.sin(a ** b)

def simulator(a, b):
    return blackbox(a, b)

custom_obj = flopt.CustomExpression(func=simulator, arg=[a, b])
print(custom_obj)

In [ ]:
# set objective function
prob += custom_obj

In [ ]:
# run solver
prob.solve(timelimit=1, msg=True)

# get best solution
print("obj value =", prob.getObjectiveValue())
print("a =", a.value())
print("b =", b.value())

When the function has a list of variables as arguments, we have the following.

In [ ]:
# problem
prob = flopt.Problem(name="CustomExpression2")

# set objective function
def blackbox(a, b):
    import math
    return math.sin(a ** b)

def simulator(x):
    return blackbox(x[0], x[1])

x = [a, b]
custom_obj = flopt.CustomExpression(func=simulator, arg=[x])
prob += custom_obj

# run solver
prob.solve(timelimit=1, msg=True)

In [ ]:
# problem
prob = flopt.Problem(name="CustomExpression3")

# set objective function
def blackbox(a, b):
    import math
    return math.sin(a ** b)

def simulator(x, y):
    return blackbox(x[0], x[1]) + blackbox(y[0], y[1])

x = [a, b]
y = [b, a]
custom_obj = flopt.CustomExpression(func=simulator, arg=[x, y])
prob += custom_obj

# run solver
prob.solve(timelimit=1, msg=True)

### [Permutation Optimization Problem](https://flopt.readthedocs.io/en/latest/tutorial/permutation.html)

```
minimize the total distance of routing the cities.
s.t.     We have to visit every city one time.
         we have the distance between all the cities.
```

In [ ]:
# We have the distance matrix D, and the number of city is N
N = 4
D = [
    [0.0, 3.0, 2.0, 1.0],
    [2.0, 0.0, 1.0, 1.0],
    [1.0, 3.0, 0.0, 4.0],
    [1.0, 1.0, 2.0, 1.0],
]

In [ ]:
import flopt

In [ ]:
prob = flopt.Problem(name="TSP")

create permutation variable

In [ ]:
perm = flopt.Variable("perm", lowBound=0, upBound=N-1, cat="Permutation")

set objective function using CustomeExpression

In [ ]:
def tsp_dist(perm):
    distance = 0
    for head, tail in zip(perm, perm[1:]+[perm[0]]):
        distance += D[head][tail]  # D is the distance matrix
    return distance

tsp_obj = flopt.CustomExpression(func=tsp_dist, arg=[perm])
prob += tsp_obj

run 2-OPT solver

In [ ]:
# run solver
prob.solve(solver="2-Opt", timelimit=3, msg=True)

# Get result
print("result = ", perm.value())

### [Max Satisfiability Problem](https://flopt.readthedocs.io/en/latest/tutorial/max_satisfiability.html)

```
maximize (c1+2*c2+3*c3+4*c4)
s.t.     c1 = x0 or x1
         c2 = x0 or not x1
         c3 = not x0 or x1
         c4 = not x0 or not x1
         x0, x1 is Binary
```

In [ ]:
import flopt

In [ ]:
prob = flopt.Problem("MaxSat", sense="Maximize")

In [ ]:
# literals
x0 = flopt.Variable("x0", cat="Binary")
x1 = flopt.Variable("x1", cat="Binary")

In [ ]:
# clauses
c1 = x0 | x1
c2 = x0 | ~x1
c3 = ~x0 | x1
c4 = ~x0 | ~x1

print(c2)

In [ ]:
# create objective function
clauses = [c1, c2, c3, c4]
weights = [1, 2, 3, 4]
obj = sum(w*c for c, w in zip(clauses, weights))  # or flopt.Dot(clauses, weights)
print(obj)

In [ ]:
# set objective function
prob += obj

# run solver
prob.solve(solver="Random", timelimit=2, msg=True)

print("value x0", x0.value())
print("value x1", x1.value())
print()
for clause in clauses:
    print(f"{clause} = {clause.value()}")

### [Expression Examples](https://flopt.readthedocs.io/en/latest/tutorial/expression_examples.html#expression-examples)

In [ ]:
import flopt

$f = \sum_i x_i$

In [ ]:
x = flopt.Variable.array('x', 4)
f = flopt.Sum(x)
print(f)

$f = \sum_i \sum_j x_i x_j$

In [ ]:
import itertools
x = flopt.Variable.array("x", 3)
f = flopt.Sum(xi * xj for xi, xj in itertools.product(x, x))
print(f)

In [ ]:
x = flopt.Variable.array("x", (3, 1))
f = flopt.Sum(x.dot(x.T))
print(f)

$f = \sum_i \left( \sum_j x_{ij} -1 \right) ^2$

In [ ]:
x = flopt.Variable.matrix("x", 2, 2)
f = flopt.Sum( (flopt.Sum(xi) - 1) ** 2 for xi in x )
print(f)

$f = \sum_{i \neq j}x_i x_j$

In [ ]:
import itertools
x = flopt.Variable.array("x", 3)
f = flopt.Sum(xi * xj for xi, xj in itertools.combinations(x, 2))
print(f)

$f = \prod_i x_i$

In [ ]:
x = flopt.Variable.array("x", 3)
f = flopt.Prod(x)
print(f)

show the calcration graph

In [ ]:
import flopt
import itertools
x = flopt.Variable.array("x", 3)
f = flopt.Sum(xi * xj for xi, xj in itertools.product(x, x))

flopt.get_dot_graph(f, "tmp.txt")
!dot tmp.txt -T png -o tmp.png

from IPython.display import Image
Image("./tmp.png")

In [ ]:
f = flopt.cos(flopt.sum(xi * xj for xi, xj in itertools.product(x, x)))

flopt.get_dot_graph(f, "tmp.txt")
!dot tmp.txt -T png -o tmp.png

from IPython.display import Image
Image("./tmp.png")

### Solver

In [ ]:
import flopt

show all available solvers 

In [ ]:
flopt.Solver_list()

In [ ]:
a = flopt.Variable("a", 0, 1, cat="Continuous")
b = flopt.Variable("b", 1, 2, cat="Continuous")

prob = flopt.Problem(name="Test")
prob += 2*a + 3*b
prob += a + b >= 1

In [ ]:
flopt.allAvailableSolvers(prob)

In [ ]:
solver = flopt.Solver(algo="auto")
solver.setParams(timelimit=1)
solver.select(prob).name

### Jacobian

$f = \prod_i x_i \Longrightarrow \frac{\partial}{\partial x_j}f = \prod_{i \neq j} x_i$

In [ ]:
x = flopt.Variable.array("x", 3)
f = flopt.Prod(x)

jac = f.jac(x)
print("jac[0] =", jac[0].getName())
print("jac[1] =", jac[1].getName())
print("jac[2] =", jac[2].getName())

In [ ]:
hess = f.hess(x)
print("jac[0] =", hess[0, 0].getName())
print("jac[1] =", hess[1, 0].getName())
print("jac[2] =", hess[2, 0].getName())

## Conversion to different formulations

### [Quadratic Programming (QP)](https://flopt.readthedocs.io/en/latest/tutorial/convert/qp.html)

In [ ]:
import flopt

# Variables
a = flopt.Variable('a', lowBound=0, upBound=1, cat='Integer')
b = flopt.Variable('b', lowBound=1, upBound=2, cat='Continuous')
c = flopt.Variable('c', lowBound=1, upBound=3, cat='Continuous')

# Problem
prob = flopt.Problem()
prob += a*a + a*b + b + c + 2
prob += a + b <= 2
prob += b - c == 3

prob.show()

In [ ]:
from flopt.convert import QpStructure
qp = QpStructure.fromFlopt(prob)

print(qp.show())

In [ ]:
print(qp.toAllEq().show())

In [ ]:
print(qp.toAllNeq().show())

### [Linear Programmnig (LP)](https://flopt.readthedocs.io/en/latest/tutorial/convert/lp.html)

In [ ]:
import flopt

# variables
a = flopt.Variable(name="a", lowBound=0, upBound=1, cat="Integer")
b = flopt.Variable(name="b", lowBound=1, upBound=2, cat="Continuous")
c = flopt.Variable(name="c", lowBound=1, upBound=3, cat="Continuous")

# problem
prob = flopt.Problem(name="LP")
prob += a + b + c + 2
prob += a + b == 2
prob += b - c <= 3

print(prob)

#### [flopt to LP](https://flopt.readthedocs.io/en/latest/tutorial/convert/lp.html#flopt-to-lp)

In [ ]:
from flopt.convert import LpStructure
lp = LpStructure.fromFlopt(prob)

print(lp.show())

In [ ]:
print(lp.toAllEq().show())

In [ ]:
print(lp.toAllNeq().toFlopt().show())

In [ ]:
print(lp.toAllNeq().show())

#### [LP to flopt](https://flopt.readthedocs.io/en/latest/tutorial/convert/lp.html#lp-to-flopt)

In [ ]:
# make Lp model
c = [1, 1, 1]
C = 2
A = [[1, 0, 1],
     [1, -1, 0]]
b = [2, 3]
lb = [1, 1, 0]
ub = [2, 3, 1]
var_types=["Binary", "Continuous", "Continuous"]

from flopt.convert import LpStructure
prob = LpStructure(c, C, A=A, b=b, lb=lb, ub=ub, types=var_types).toFlopt()

prob.show()

### [Ising](https://flopt.readthedocs.io/en/latest/tutorial/convert/ising.html)

In [ ]:
import flopt

# Variables
a = flopt.Variable('a', cat='Spin')
b = flopt.Variable('b', cat='Spin')

# Problem
prob = flopt.Problem()
prob += 1 - a * b - a

print(prob)

#### [flopt to Ising](https://flopt.readthedocs.io/en/latest/tutorial/convert/ising.html#flopt-to-ising)

In [ ]:
import flopt

# Variables
a = flopt.Variable('a', cat='Spin')
b = flopt.Variable('b', cat='Binary') # Binary variable

# Problem
prob = flopt.Problem()
prob += 1 - a * b - a

print(prob)

In [ ]:
from flopt.convert import IsingStructure
ising = IsingStructure.fromFlopt(prob)

print(ising.show())

#### [Convert to QUBO](https://flopt.readthedocs.io/en/latest/tutorial/convert/ising.html#convert-to-qubo)

In [ ]:
ising.toQubo()    # convert ising to QUBO

print(ising.toQubo().toFlopt().show())  # for show cleary ising.toQubo()

In [ ]:
flopt.allAvailableSolvers(prob)

#### [Ising to flopt](https://flopt.readthedocs.io/en/latest/tutorial/convert/ising.html#ising-to-flopt)

In [ ]:
# make ising model
J = [[0, 1],
     [0, 0]]
h = [1, 0]
C = 1

from flopt.convert import IsingStructure
prob = IsingStructure(J, h, C).toFlopt()

prob.show()

### [Quadratic Unconstrainted Binary Programming (QUBO)](https://flopt.readthedocs.io/en/latest/tutorial/convert/qubo.html)

In [ ]:
import flopt

# Variables
a = flopt.Variable('a', cat='Binary')
b = flopt.Variable('b', cat='Binary')

# Problem
prob = flopt.Problem()
prob += 1 - a * b - a

print(prob)

#### [flopt to QUBO](https://flopt.readthedocs.io/en/latest/tutorial/convert/qubo.html#flopt-to-qubo)

In [ ]:
from flopt.convert import QuboStructure
qubo = QuboStructure.fromFlopt(prob)

print(qubo.show())

#### [QUBO to flopt](https://flopt.readthedocs.io/en/latest/tutorial/convert/qubo.html#qubo-to-flopt)

In [ ]:
# make ising
Q = [[-1, -1],
     [0, 0]]
C = 1.0

from flopt.convert import QuboStructure
prob = QuboStructure(Q, C).toFlopt()

print(prob)

```
minimize  a * b * c + 2
s.t.      a * b == 2
          0 <= a <= 1, a is integer
          1 <= b <= 2, b is continuous
          1 <= c <= 3, c is continuous
 ```

In [ ]:
a = flopt.Variable(name="a", lowBound=0, upBound=1, cat="Integer")
b = flopt.Variable(name="b", lowBound=1, upBound=2, cat="Continuous")
c = flopt.Variable(name="c", lowBound=1, upBound=3, cat="Continuous")

In [ ]:
prob = flopt.Problem()
prob += a * b * c + 2
prob += a * c == 2
prob.show()

# docstrings

## Solvers

### ScipySearch

In [ ]:
import flopt

# Variables
a = flopt.Variable("a", lowBound=-2, upBound=1, cat="Integer")
b = flopt.Variable("b", lowBound=1, upBound=4, cat="Continuous")
c = flopt.Variable("c", lowBound=0, upBound=3, cat="Continuous")

# Problem
prob = flopt.Problem()
prob += a*a + a*b + b + c + 2
prob += a + b >= 2
prob += b - c == 3

prob.solve(solver="Scipy", msg=True)

print(flopt.Value([a, b, c]))

### PulpSearch

In [ ]:
import flopt

# Variables
a = flopt.Variable("a", lowBound=0, upBound=1, cat="Integer")
b = flopt.Variable("b", lowBound=1, upBound=2, cat="Continuous")
c = flopt.Variable("c", lowBound=-1, upBound=3, cat="Continuous")

# Problem
prob = flopt.Problem()
prob += a + b + c + 2
prob += a + b >= 2
prob += b - c >= 3

prob.solve(solver="Pulp", msg=True)

In [ ]:
solver = flopt.Solver("Pulp")

import pulp
glpk_solver = pulp.GLPK_CMD()
solver.setParams(solver=glpk_solver)
prob.solve(solver, msg=True)

### ScipyMilpSearch

In [ ]:
import flopt

# Variables
a = flopt.Variable("a", lowBound=0, upBound=1, cat="Integer")
b = flopt.Variable("b", lowBound=1, upBound=2, cat="Continuous")
c = flopt.Variable("c", lowBound=-1, upBound=3, cat="Continuous")

# Problem
prob = flopt.Problem()
prob += a + b + c + 2
prob += a + b >= 2
prob += b - c >= 3

prob.solve(solver="ScipyMilp", msg=True)

### CvxoptSearch

In [ ]:
import flopt

x = flopt.Variable("x", lowBound=-1, upBound=1, cat="Continuous")
y = flopt.Variable("y", lowBound=-1, upBound=1, cat="Continuous")

prob = flopt.Problem()
prob += 2*x*x + x*y + y*y + x + y
prob += x >= 0
prob += y >= 0
prob += x + y == 1

status, log = prob.solve(solver="CvxoptQp", msg=True)

print("obj =", flopt.Value(prob.obj))
print("x =", flopt.Value(x))
print("y =", flopt.Value(y))

### OptunaCmaEsSearch, OptunaTPESearch, HyperoptSearch, SFLA, RandomSearch

In [ ]:
import flopt

x = flopt.Variable("x", lowBound=-1, upBound=1, cat="Continuous")
y = flopt.Variable("y", lowBound=-1, upBound=1, cat="Continuous")

prob = flopt.Problem()
prob += 2*x*x + x*y + y*y + x + y

status, log = prob.solve(solver="OptunaCmaEs", msg=True, timelimit=1)

print("obj =", flopt.Value(prob.obj))
print("x =", flopt.Value(x))
print("y =", flopt.Value(y))

In [ ]:
import flopt

x = flopt.Variable("x", lowBound=-1, upBound=1, cat="Continuous")
y = flopt.Variable("y", lowBound=-1, upBound=1, cat="Continuous")

prob = flopt.Problem()
prob += 2*x*x + x*y + y*y + x + y

status, log = prob.solve(solver="OptunaTPE", msg=True, timelimit=1)

print("obj =", flopt.Value(prob.obj))
print("x =", flopt.Value(x))
print("y =", flopt.Value(y))

In [ ]:
import flopt

x = flopt.Variable("x", lowBound=-1, upBound=1, cat="Continuous")
y = flopt.Variable("y", lowBound=-1, upBound=1, cat="Continuous")

prob = flopt.Problem()
prob += 2*x*x + x*y + y*y + x + y

status, log = prob.solve(solver="Hyperopt", msg=True, timelimit=1)

print("obj =", flopt.Value(prob.obj))
print("x =", flopt.Value(x))
print("y =", flopt.Value(y))

In [ ]:
import flopt

x = flopt.Variable("x", lowBound=-1, upBound=1, cat="Continuous")
y = flopt.Variable("y", lowBound=-1, upBound=1, cat="Continuous")

prob = flopt.Problem()
prob += 2*x*x + x*y + y*y + x + y

status, log = prob.solve(solver="SFLA", msg=True, timelimit=1)

print("obj =", flopt.Value(prob.obj))
print("x =", flopt.Value(x))
print("y =", flopt.Value(y))

In [ ]:
import flopt

x = flopt.Variable("x", lowBound=1, upBound=1, cat="Continuous")
y = flopt.Variable("y", lowBound=-1, upBound=1, cat="Continuous")

prob = flopt.Problem()
prob += 2*x*x + x*y + y*y + x + y

status, log = prob.solve(solver="Random", msg=True, timelimit=1)

print("obj =", flopt.Value(prob.obj))
print("x =", flopt.Value(x))
print("y =", flopt.Value(y))

### AutoSearch

In [ ]:
import flopt

# Variables
a = flopt.Variable("a", lowBound=0, upBound=1, cat="Integer")
b = flopt.Variable("b", lowBound=1, upBound=2, cat="Continuous")
c = flopt.Variable("c", lowBound=1, upBound=3, cat="Continuous")

prob = flopt.Problem(name="Test")
prob += 2*(3*a+b)*c**2+3

In [ ]:
prob.solve(solver="auto", timelimit=10, msg=True)

In [ ]:
solver = flopt.Solver(algo="auto")
solver.setParams({"timelimit": 10})
solver = solver.select(prob)
print(solver.name)

# Case study

## newton's method

In [ ]:
import flopt
from flopt import Value

# define variable with initial value
x = flopt.Variable("x", ini_value=5)

# define f
f = x * x - 2

# obtain jacobian function
jac = f.jac([x])

for roop in range(10):
    # update x
    x.setValue( (x - f / jac).value() )
    print(f'roop {roop:<4d} x = {x.value()}')

In [ ]:
import flopt
from flopt import Value

# define f
def f(x):
    return x * x - 2

# define variable with initial value
x = flopt.Variable("x", ini_value=5)

# obtain jacobian function
jac = f(x).jac([x])

for roop in range(10):
    # update xk
    x.setValue( (x - f(x) / jac).value() )
    print(f'roop {roop:<4d} x = {x.value()}')